<a href="https://colab.research.google.com/github/Huhan001/tensorflowcertification/blob/main/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow_datasets


In [ ]:
#import libaries that im going to use.
#tensorflow
import tensorflow as tf
import tensorflow_datasets as tfds 

#regular
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

#to improve progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)


In [ ]:
#loading data from tensorflow dataset

dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True) #- data fetched 

train_dataset, test_dataset = dataset['train'], dataset['test']

print(train_dataset)

In [ ]:
#class names are not included in the data so i need to store them in a different enviroment. 
class_names = metadata.features['label'].names
print("class names are: {}".format(class_names))

In [ ]:
#data exploration
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))


In [ ]:
#normalization in images

def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)


#caching images makes them live in the memory and make training fater other than loading them from the database

train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()


In [ ]:
#time to visualize our data

for image, label in test_dataset.take(9):
  break

image = image.numpy().reshape((28,28))
#plt.figure()
plt.imshow(image)
#plt.colorbar()
#plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(train_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.cache().batch(BATCH_SIZE)

model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)